## in this Demo we will use Semantic Kernal to create chatbot for a VOD service to recommend movies to the users based on Genre, Cast and movie Overview.

In [28]:
from typing import List, Optional, TypedDict, Annotated
from typing import Annotated
from semantic_kernel.functions import kernel_function
import os
import openai
import ast
import json
import random
from dotenv import load_dotenv, find_dotenv
from openai import AzureOpenAI
import asyncio
from semantic_kernel import Kernel
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.function_call_behavior import FunctionCallBehavior
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions.kernel_arguments import KernelArguments
import asyncio
import pandas as pd
import numpy as np
import logging
from semantic_kernel import Kernel
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.function_call_behavior import FunctionCallBehavior
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)

In [29]:


credits=pd.read_csv('the-movies-dataset/credits.csv')
movies=pd.read_csv('the-movies-dataset/movies_metadata.csv')

import unicodedata
import re
def clean_data(value):
    value =  unicodedata.normalize('NFD', value).encode('ascii', 'ignore').decode('ascii')
    x= str.lower(re.sub(r"[^a-zA-Z0-9]+", '_', value.strip().replace(" ","_")))
    if(x!='_'):
        return x 
    else:
        return " " 
    
credits['id']=credits['id'].astype('str')
training_df=(credits.merge(movies,on='id'))
training_df['popularity']=training_df['popularity'].astype('float')
training_df=training_df[['title','cast','genres','overview','popularity']]
training_df['overview']=training_df['overview'].str.lower()
training_df['cast']=training_df['cast'].apply(lambda s: list(ast.literal_eval(s)))
training_df['cast'] = training_df['cast'].map(lambda x: x[:3] if len(x) >=4 else x)
training_df['cast']=training_df['cast'].apply((lambda cast : [ clean_data(actor['name'])  for actor in cast  if actor != '' and actor != ' ' ]))
training_df['genres']=training_df['genres'].apply(lambda s: list(ast.literal_eval(s)))
training_df['genres']=training_df['genres'].apply((lambda genres : [ clean_data(genre['name'])  for genre in genres]))


C:\Users\v-akhrisat\AppData\Local\Temp\ipykernel_20608\3260591196.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies=pd.read_csv('the-movies-dataset/movies_metadata.csv')


In [43]:


class Recommendations:
 
       
    @kernel_function(
        description="use this function only when user ask you for a movie recomendation based on genre, cast or  overview",
        name="get_movie_recommendation",
    )     
    async def get_movie_recommendation(self,genre:Annotated[str, "The movie genres like Sci-Fi, Drama, Romance"],
                                 cast:Annotated[List[str], "The movie cast could be more than one actor"]
                                 , overview:Annotated[str, "part of movie description like War, Race,Family..."]):
   
         movie_name=None
         if genre is not None:
            genre_filter = training_df['genres'].map(lambda u: clean_data(genre)  in u)
            if cast is not None:
                cast= [ clean_data(actor)  for actor in cast  if actor != '' and actor != ' ' ]
                cast_filter = training_df['cast'].map(lambda c: set(cast).issubset(c))
            if overview is not None:
                overview_filter =training_df['overview'].str.contains(overview)
    
    
            if cast is not None and overview is not None:    
              result=training_df[genre_filter & cast_filter &overview_filter]
            elif  cast is None and overview is not None:  
                 result=training_df[genre_filter  &overview_filter]
            elif  cast is not None and overview is  None:  
                 result=training_df[genre_filter  &cast_filter   ]
            else : "
                 result=training_df[genre_filt"er]
    
            if result.shape[0]>=1:
                   movie_name =result.sort_values(by=['popularity'],ascending=False).head(1)['title'].item()
         if movie_name  is not None:
                print(movie_name)   
                return { "movie_name": movie_name}
         else:
                return  { "movie_name":'we could not find any movie'}



SyntaxError: unterminated string literal (detected at line 28) (2806366464.py, line 28)

In [44]:
_=load_dotenv(find_dotenv()) 

AZURE_ENDPOINT =os.environ["AZURE_OPENAI_ENDPOINT"]
API_KEY= os.environ['OPENAI_API_KEY']
API_VERSION=os.environ['api_version']
OPENAI_DEPLOYMENT=os.environ["OPENAI_DEPLOYMENT"]


In [ ]:

async def main():
    # Initialize the kernel
    kernel = Kernel()
 
    kernel.add_service(AzureChatCompletion(
      deployment_name=OPENAI_DEPLOYMENT,
      api_key=API_KEY,
      api_version=  API_VERSION,
      endpoint=AZURE_ENDPOINT
    ))


    # Set the logging level for  semantic_kernel.kernel to DEBUG.
    logging.basicConfig(
        format="[%(asctime)s - %(name)s:%(lineno)d - %(levelname)s] %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S",
    )
    logging.getLogger("kernel").setLevel(logging.DEBUG)

    kernel.add_plugin(
        Recommendations(),
        plugin_name="recommendations",
    )

    chat_completion : AzureChatCompletion = kernel.get_service(type=ChatCompletionClientBase)

    # Enable planning
    execution_settings = AzureChatPromptExecutionSettings(tool_choice="auto")
    execution_settings.function_call_behavior = FunctionCallBehavior.EnableFunctions(auto_invoke=True, filters={})

    # Create a history of the conversation
    history = ChatHistory()

    # Initiate a back-and-forth chat
    userInput = None
    while True:
        # Collect user input
        userInput = input("User > ")

        # Terminate the loop if the user says "exit"
        if userInput == "exit":
            break

        # Add user input to the history
        history.add_user_message(userInput)

        # Get the response from the AI
        result = (await chat_completion.get_chat_message_contents(
            chat_history=history,
            settings=execution_settings,
            kernel=kernel,
            arguments=KernelArguments(),
        ))[0]

        # Print the results
        print("Assistant > " + str(result))

        # Add the message from the agent to the chat history
        history.add_message(result)

await main()

C:\Users\v-akhrisat\AppData\Local\Temp\ipykernel_20608\474962906.py:29: DeprecationWarning: Use the `FunctionChoiceBehavior` `Auto` class method instead.
  execution_settings.function_call_behavior = FunctionCallBehavior.EnableFunctions(auto_invoke=True, filters={})
[2024-08-08 00:25:36 - semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.open_ai_prompt_execution_settings:103 - WARNING] The `function_call_behavior` parameter is deprecated. Please use the `function_choice_behavior` parameter instead.


User >  recommend action moive


Assistant > I recommend the action movie **"Wonder Woman."** It's filled with thrilling sequences and an engaging storyline. Enjoy!


User >  Recommend Drama movie


Assistant > I recommend the drama movie **"Gone Girl."** It's a captivating and intense film with a complex plot. Enjoy watching!
